In [1]:
### Imports ###
import os
from typing import List, Tuple, Union, Dict
from classes.client import Client
from classes.coordinator_utils import select_common_features_variables, \
    compute_beta, create_beta_mask

import numpy as np
import pandas as pd
import time

In [2]:
### Helper Functions ###
### Just run this, these functions are needed in various places ###

### Define the client class ###
class ClientWrapper:
    """
    Holds all information necessary for the simulation run to work.
    Defines the input data of the client, it's name and if it should be
    considered as the coordinator
    """
    def __init__(self, id: str, input_folder: str, coordinator: bool = False):
        self.id = id
        self.input_folder = input_folder
        self.is_coordinator = coordinator
        self.client_class: Client # initiated later
        self.data_corrected: pd.DataFrame # initiated later

def _check_consistency_clientwrappers(clientWrappers: List[ClientWrapper]) -> None:
    """
    Checks for a list of clients if they were created correctly
    Raises a ValueError in case of inconsistencies
    Checks:
        1. If exactly one coordinator exists
    """
    coord = False
    for clientWrapper in clientWrappers:
        if coord and clientWrapper.is_coordinator:
            raise ValueError("More than one coordinator was defined, please check "+\
                            "the code defining the clients")
        if clientWrapper.is_coordinator:
            coord = True
    if not coord:
        raise ValueError("No client instance is a coordinator, please designate "+\
                        "any client as a coordinator")

def _compare_central_federated_dfs(name:str,
                                   central_df: pd.DataFrame,
                                   federated_df: pd.DataFrame,
                                   intersection_features: List[str]) -> None:
    """
    Compares two dataframes for equality. First checks that index and columns
    are the same, then analyses the element wise differences.
    See the analyse_diff_df function for more details on the difference analysis.
    If both dataframes contain a NaN value at the same position, this is considered
    as equal (0 as difference).
    Args:
        name: Name used for printing
        central_df: The central dataframe
        federated_df: The federated dataframe
        intersection_features: The features that are common to both dataframes
    """
    central_df = central_df.sort_index(axis=0).sort_index(axis=1)
    federated_df = federated_df.sort_index(axis=0).sort_index(axis=1)
    print(f"_________________________Analysing: {name}_________________________")
    ### compare columns and index ###
    failed = False
    if not central_df.columns.equals(federated_df.columns):
        print(f"Columns do not match for central_df and federated_df")
        union_cols = central_df.columns.union(federated_df.columns)
        intercept_cols = central_df.columns.intersection(federated_df.columns)
        print(f"Union-Intercept of columns: {union_cols.difference(intercept_cols)}")
        failed = True
    if not central_df.index.equals(federated_df.index):
        print(f"Rows do not match for central_df and federated_df")
        union_rows = central_df.index.union(federated_df.index)
        intercept_rows = central_df.index.intersection(federated_df.index)
        print(f"Union-Intercept of rows: {union_rows.difference(intercept_rows)}")
        failed = True
    if failed:
        print(f"_________________________FAILED: {name}_________________________")

    df_diff = (central_df - federated_df).abs()
    print(f"Max difference: {df_diff.max().max()}")
    print(f"Mean difference: {df_diff.mean().mean()}")
    print(f"Max diff at position: {df_diff.idxmax().idxmax()}")

    df_diff_intersect = df_diff.loc[intersection_features]
    print(f"Max difference in intersect: {df_diff_intersect.max().max()}")
    print(f"Mean difference in intersect: {df_diff_intersect.mean().mean()}")
    print(f"Max diff at position in intersect: {df_diff_intersect.idxmax().idxmax()}")

def _concat_federated_results(clientWrappers: List[ClientWrapper],
                              samples_in_columns=True) -> Tuple[pd.DataFrame, List[str]]:
    """
    Concatenates the results of the federated clients into one dataframe
    Also checks which features are common to all clients
    and returns them
    Args:
        clientWrappers: List of ClientWrapper instances, containing the data
            in the data_corrected attribute
        samples_in_columns: If True, the samples are in the columns, if False
            they are in the rows. For expression files this is true.
            This decides how to aggregate the dataframes
    Returns:
        merged_df: The merged dataframe containing the data of all clients
        intersection_features: The features that are common to all clients
    """
    merged_df = None
    intersection_features = set()
    for clientWrapper in clientWrappers:
        # get the data in the correct format
        if not hasattr(clientWrapper, "data_corrected") or \
            clientWrapper.data_corrected is None:
            raise ValueError("No data was found in the clientWrappers")
        corrected_data = clientWrapper.data_corrected
        if not samples_in_columns:
            corrected_data = corrected_data.T

        cleaned_corrected_features = set(corrected_data.dropna().index)
        # initialize the merged_df
        if merged_df is None:
            merged_df = corrected_data
            intersection_features = cleaned_corrected_features
            continue

        # merge the data
        merged_df = pd.concat([merged_df, corrected_data], axis=1)
        intersection_features = intersection_features.intersection(cleaned_corrected_features)

    # final check
    if merged_df is None:
        raise ValueError("No data was found in the clientWrappers")
    # reverse the Transpose if necessary
    if not samples_in_columns:
        merged_df = merged_df.T
    return merged_df, list(intersection_features)


## DATA

In [ ]:
### This part defines the data used. A ClientWrapper class is used to      ###
### describe all cohorts.                                                  ###
### Comment in the wanted data or add a new block for new data. No other   ###
### changes are necessary.                                                 ###

In [ ]:
#################### Microarray Data ####################
# First define the basefolder where all files are located
base_dir = os.path.join("..")
# Go back to the git repos root dir
datafolder = "microarray"
base_dir = os.path.join(base_dir, "evaluation_data", "microarray", "before")

# List of cohort names
cohort_names = [
    'GSE38666',  # Client 1 (Coordinator)
    'GSE14407',  # Client 2
    'GSE6008',   # Client 3
    'GSE40595',  # Client 4
    'GSE26712',  # Client 5
    'GSE69428',  # Client 6
]
output_name = "microarray"
central_filename = "central_corrected_UNION.tsv"

In [5]:
# #################### Proteomics Data ####################

# # First define the basefolder where all files are located
# base_dir = os.path.join("..")
# # Go back to the git repos root dir
# datafolder = "proteomics"
# base_dir = os.path.join(base_dir, "evaluation_data", datafolder, "before")

# # List of cohort names
# cohort_names = [
#     'lab_A',  # Client 1 (Coordinator)
#     'lab_B',  # Client 2
#     'lab_C',  # Client 3
#     'lab_D',  # Client 4
#     'lab_E',  # Client 5
# ]
# output_name = "proteomics"
# central_filename = "intensities_log_Rcorrected_UNION.tsv"

In [ ]:
# #################### Proteomics Multibatch Data ####################

# # First define the basefolder where all files are located
# base_dir = os.path.join("..")
# # Go back to the git repos root dir
# datafolder = "proteomics_multibatch"
# base_dir = os.path.join(base_dir, "evaluation_data", datafolder, "before")

# # List of cohort names
# cohort_names = [
#     'center1', # Client 1 (Coordinator)
#     'center2', # Client 2
#     'center3' # Client 3
# ]
# output_name = "proteomics_multibatch"
# central_filename = "intensities_log_Rcorrected_UNION.tsv"

In [7]:
# ################# Microbiome Data ####################
# # First define the basefolder where all files are located
# base_dir = os.path.join("..")
# # Go back to the git repos root dir
# datafolder = "microbiome"
# base_dir = os.path.join(base_dir, "evaluation_data", datafolder, "before")

# # List of cohort names
# cohort_names = [
#     'PRJEB27928',  # Client 1 (Coordinator)
#     'PRJEB6070',   # Client 2
#     'PRJNA429097', # Client 3
#     'PRJEB10878',  # Client 4
#     'PRJNA731589', # Client 5
# ]
# output_name = "microbiome"
# central_filename = "normalized_logmin_counts_5centers_Rcorrected.tsv"

In [8]:
# ################### Simulation Data ###################

# # First define the basefolder where all files are located
# base_dir = os.path.join("..")
# # Go back to the git repos root dir

# # There are three different setups, activate the wanted one here accordingly, deactivate the others:
# # BALANCED
# # datafolder = os.path.join("simulated", "balanced")
# # base_dir = os.path.join(base_dir, "evaluation_data", "simulated", "balanced", "before")
# # output_name = "simulated_balanced"
# # # MILD IMBALANCED
# # datafolder = os.path.join("simulated", "mild_imbalanced")
# # base_dir = os.path.join(base_dir, "evaluation_data", "simulated", "mild_imbalanced", "before")
# # output_name = "simulated_mild_imbalanced"
# # # STRONG IMBALANCED
# datafolder = os.path.join("simulated", "strong_imbalanced")
# base_dir = os.path.join(base_dir, "evaluation_data", "simulated", "strong_imbalanced", "before")
# output_name = "simulated_strong_imbalanced"

# # List of cohort names
# cohort_names = [
#     'lab1',  # Client 1 (Coordinator)
#     'lab2',  # Client 2
#     'lab3',  # Client 3
# ]
# central_filename = "intensities_R_corrected.tsv" # is the same for all simulated setups

In [9]:

# Initialize clientWrappers list
clientWrappers: List[ClientWrapper] = []

# Iterate over cohort names and create ClientWrapper instances
for i, cohortname in enumerate(cohort_names):
    clientWrappers.append(ClientWrapper(
        id=cohortname,
        input_folder=os.path.join(base_dir, cohortname),
        coordinator=(i == 0)  # Set the first client as coordinator
    ))

# Double check that we only have one coordinator
_check_consistency_clientwrappers(clientWrappers)


## Analysis

In [10]:
# measure time for all clients
time_tracker = {}

In [11]:
###                                  INFO                                  ###
### The following code blocks run the simulation. They are divided into    ###
### multiple logical blocks to ease the use                                ###

In [12]:
### SIMULATION: all: initial ###
### Initial reading of the input folder

send_feature_variable_batch_info = list()
for clientWrapper in clientWrappers:
    # define the client class
    cohort_name = clientWrapper.id
    client = Client()
    client.config_based_init(clientname = cohort_name,
                             input_folder = clientWrapper.input_folder,
                             use_hashing = False)
    clientWrapper.client_class = client
    assert isinstance(client.hash2feature, dict)
    assert isinstance(client.hash2variable, dict)
    batch_feature_presence_info: Dict[str, List[str]] = client.get_batch_feature_presence_info()
    send_feature_variable_batch_info.append((cohort_name,
                                        list(client.hash2variable.keys()),
                                        client.position,
                                        batch_feature_presence_info))


Got the following config:
{'flimmaBatchCorrection': {'batch_col': 'batch', 'covariates': ['Pyr'], 'data_filename': 'intensities_log_UNION.tsv', 'design_filename': 'design.tsv', 'design_separator': '\t', 'expression_file_flag': True, 'index_col': 'rowname', 'min_samples': 0, 'normalizationMethod': None, 'position': 0, 'separator': '\t', 'smpc': False}}
Opening dataset ../evaluation_data/proteomics_multibatch/before/center1/intensities_log_UNION.tsv
Shape of rawdata(expr_file): (2910, 47)
finished loading data, shape of data: (2910, 47), num_features: 2910, num_samples: 47
Checking batch:  lab_A
self.design has these batch values: ['lab_A' 'lab_B']
Got these batch indices: ['Ref8537_QC1_20230414_1', 'Ref8537_QC2_20230414_1', 'Ref8537_QC3_20230414_1', 'Ref8537_QC4_20230414_1', 'Ref8537_S4_20230414', 'Ref8537_S8_20230414', 'Ref8537_S11_20230414', 'Ref8537_S18_20230414', 'Ref8537_S21_20230414', 'Ref8537_S26_20230414', 'Ref8537_S35_20230414', 'Ref8537_S37_20230414', 'Ref8537_S45_20230414', '

In [13]:
### SIMULATION: Coordinator: global_feature_selection ###
### Aggregate the features and variables

# obtain and safe common genes and indices of design matrix
# wait for each client to send the list of genes they have
# also memo the feature presence matrix and feature_to_cohorts

broadcast_features_variables = tuple()
for clientWrapper in clientWrappers:
    if clientWrapper.is_coordinator:

        time_tracker["Coordinator"] = time.time()

        global_feature_names, global_variables, feature_presence_matrix, cohorts_order = \
            select_common_features_variables(
                feature_variable_batch_info=send_feature_variable_batch_info,
                min_clients=1,
                default_order=cohort_names
            )

        # memo the feature presence matrix and feature_to_cohorts
        broadcast_features_variables = global_feature_names, global_variables, cohorts_order
        print(f"Got this final cohort order: {cohorts_order}")
        end_time = time.time()
        time_tracker["Coordinator"] = end_time - time_tracker["Coordinator"]


INFO: All cohorts: [['center1|lab_A', 'center1|lab_B'], ['center2|lab_C', 'center2|lab_D'], ['center3|lab_E']]
Using given specific client order: ['center1', 'center2', 'center3']
INFO: Cohorts order: ['center1|lab_A', 'center1|lab_B', 'center2|lab_C', 'center2|lab_D', 'center3|lab_E']
Got this final cohort order: ['center1|lab_A', 'center1|lab_B', 'center2|lab_C', 'center2|lab_D', 'center3|lab_E']


In [14]:
### SIMULATION: All: validate ###
### Expand data to fullfill the global format. Also performs consistency checks
cols = []
for clientWrapper in clientWrappers:

    time_tracker[clientWrapper.id] = time.time()

    global_feauture_names_hashed, global_variables_hashed, cohorts_order = \
        broadcast_features_variables
    client = clientWrapper.client_class
    client.validate_inputs(global_variables_hashed)
    client.set_data(global_feauture_names_hashed)

    err = client.create_design(cohorts_order)
    cols.append(client.design.columns)
        # we choose the last batch of the last cohort as reference batch
    if err:
        raise ValueError(err)

    end_time = time.time()
    time_tracker[clientWrapper.id] = end_time - time_tracker[clientWrapper.id]
for col in cols:
    print(col)
    assert cols[0].equals(col)

Client center1: Inputs validated.
Number of features available in this client: 2910
Number of features given globally: 3034
Number of features only available on this client: 0
Number of features available in other clients but not this client: 124
Adding 124 extra global features
Design matrix created:
                                        intercept  Pyr  center1|lab_A  \
file                                                                    
Ref8537_QC1_20230414_1                        1.0    1              1   
Ref8537_QC2_20230414_1                        1.0    1              1   
Ref8537_QC3_20230414_1                        1.0    0              1   
Ref8537_QC4_20230414_1                        1.0    0              1   
Ref8537_S4_20230414                           1.0    1              1   
Ref8537_S8_20230414                           1.0    1              1   
Ref8537_S11_20230414                          1.0    1              1   
Ref8537_S18_20230414                    

In [15]:
### Simulatuion: Coordinator: create design mask based on feature presence matrix ###
### Create the mask for the design matrix based on the feature presence matrix
### that will be used for the beta computation
for clientWrapper in clientWrappers:
    if clientWrapper.is_coordinator:
        start_time = time.time()
        client = clientWrapper.client_class

        n=len(client.feature_names)
        k=client.design.shape[1]

        global_mask = create_beta_mask(feature_presence_matrix, n, k)
        print(f"Shape of global mask: {global_mask.shape}")
        print(f"Head of mask: {global_mask[:5]}")
        # memo the global mask

        end_time = time.time()
        time_tracker["Coordinator"] += end_time - start_time


INFO: Global mask true count:
1844
Shape of global mask: (3034, 6)
Head of mask: [[False False False False False  True]
 [False False  True  True  True  True]
 [False False  True False  True  True]
 [False False False False False False]
 [False False False False False  True]]


In [16]:
### SIMULATION: All: prepare for compute_XtX_XtY ###

for clientWrapper in clientWrappers:
    start_time = time.time()

    client = clientWrapper.client_class
    client.sample_names = client.design.index.values

    # Error check if the design index and the data index are the same
    # we check by comparing the sorted indexes
    client._check_consistency_designfile()

    # Extract only relevant (the global) features and samples
    client.data = client.data.loc[client.feature_names, client.sample_names]
    client.n_samples = len(client.sample_names)

    end_time = time.time()
    time_tracker[clientWrapper.id] += end_time - start_time

In [17]:
### SIMULATION: All: compute_XtX_XtY ###
### Compute XtX and XtY and share it
send_XtX_XtY_list: List[List[np.ndarray]] = list()
for clientWrapper in clientWrappers:
    start_time = time.time()

    client = clientWrapper.client_class

    # compute XtX and XtY
    XtX, XtY, err = client.compute_XtX_XtY()
    if err != "":
        raise ValueError(err)

    # send XtX and XtY
    send_XtX_XtY_list.append([XtX, XtY])

    end_time = time.time()
    time_tracker[clientWrapper.id] += end_time - start_time

In [18]:
### SIMULATION: Coordinator: compute_beta
### Compute the beta values and broadcast them to the others
broadcast_betas = None # np.ndarray of shape num_features x design_columns

for clientWrapper in clientWrappers:
    if clientWrapper.is_coordinator:

        start_time = time.time()

        client = clientWrapper.client_class
        beta = compute_beta(XtX_XtY_list=send_XtX_XtY_list,
                            n=len(client.feature_names),
                            k=client.design.shape[1],
                            global_mask=global_mask)

        # send beta to clients so they can correct their data
        broadcast_betas = beta

        end_time = time.time()
        time_tracker["Coordinator"] += end_time - start_time

INFO: Random XtX entry: [[ 95.  47.   0. -24.  -1.   0.]
 [ 47.  47.   0. -12.  -1.   0.]
 [  0.   0.  48.  24.  24.  24.]
 [-24. -12.  24.  24.  24.  24.]
 [ -1.  -1.  24.  24.  47.  24.]
 [  0.   0.  24.  24.  24.  48.]]
INFO: All XtX entries are the same?: False
INFO: Shape of beta: (3034, 6)
INFO: Number of pseudo inverses: 0


In [19]:
### SIMULATION: All: include_correction
### Corrects the individual data
for clientWrapper in clientWrappers:

    start_time = time.time()

    client = clientWrapper.client_class

    # remove the batch effects in own data and safe the results
    client.remove_batch_effects(beta)
    print(f"DEBUG: Shape of corrected data: {client.data_corrected.shape}")

    end_time = time.time()
    time_tracker[clientWrapper.id] += end_time - start_time

    # As this is a simulation we don't save the corrected data to csv, instead
    # we save it as a variable to the clientwrapper
    clientWrapper.data_corrected = client.data_corrected
    # client.data_corrected.to_csv(os.path.join(os.getcwd(), "mnt", "output", "only_batch_corrected_data.csv"),
    #                                 sep=self.load("separator"))
    # client.data_corrected_and_raw.to_csv(os.path.join(os.getcwd(), "mnt", "output", "all_data.csv"),
    #                              sep=self.load("separator"))
    # with open(os.path.join(os.getcwd(), "mnt", "output", "report.txt"), "w") as f:
    #     f.write(client.report)


Start remove_batch_effects
Shape of corrected data after correction: (3034, 47)
Amount of index found in hash2feature: 2910/2910
remove batch final corrected data shape: (2910, 47)
DEBUG: Shape of corrected data: (2910, 47)
Start remove_batch_effects
Shape of corrected data after correction: (3034, 47)
Amount of index found in hash2feature: 2900/2900
remove batch final corrected data shape: (2900, 47)
DEBUG: Shape of corrected data: (2900, 47)
Start remove_batch_effects
Shape of corrected data after correction: (3034, 24)
Amount of index found in hash2feature: 2363/2363
remove batch final corrected data shape: (2363, 24)
DEBUG: Shape of corrected data: (2363, 24)


In [20]:
# print the time tracker for the coordinator
print(f"Time tracker for coordinator, ms: {round(time_tracker['Coordinator']*1000, 2)}")

# print the time tracker for the clients
for clientWrapper in clientWrappers:
    print(f"Time tracker for {clientWrapper.id}, ms: {round(time_tracker[clientWrapper.id]*1000, 2)}")

Time tracker for coordinator, ms: 203.23
Time tracker for center1, ms: 235.53
Time tracker for center2, ms: 224.88
Time tracker for center3, ms: 265.94


In [21]:
###                                  INFO                                  ###
###                            SIMULATION IS DONE                          ###
### The simulation is done. The corrected data is saved in the             ###
### clientWrapper instances. Now we analyse the data by comparing to the   ###
### calculated centralized corrected data.                                 ###


In [22]:
federated_df, intersect_features = _concat_federated_results(clientWrappers, samples_in_columns=True)

In [23]:
### SAVE THE RESULTS ###
#federated_df.to_csv(os.path.join("..", "evaluation_data", datafolder, "after", "FedSim_corrected_data.tsv"), sep="\t")

In [24]:
### Concat the federated data and read in the centralized data ###
base_dir = os.path.join("..", "evaluation_data")
central_df_path = os.path.join(base_dir, datafolder, "after", central_filename)
central_df = pd.read_csv(central_df_path, sep="\t", index_col=0)
_compare_central_federated_dfs(output_name, central_df, federated_df, intersect_features)

_________________________Analysing: proteomics_multibatch_________________________
Max difference: 1.5631940186722204e-13
Mean difference: 3.16976089953702e-14
Max diff at position: 20230516_EXPL1_LiSc_SA_S610_RT_40_s80
Max difference in intersect: 1.1723955140041653e-13
Mean difference in intersect: 3.33913166227243e-14
Max diff at position in intersect: CVT09_s13_X004
